In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jinkim89","key":"097591ab6ed3746b82c87eb8ec6edb3e"}'}

In [2]:
ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        107           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        212           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        576           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       3528           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      28555            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       7297           False  
connectx

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.8MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 26.7MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.15MB/s]


In [5]:
!ls

gender_submission.csv  kaggle.json  sample_data  test.csv  train.csv


In [9]:
batch_size = 32

In [230]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [176]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [150]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [142]:
cat_dict={}
def categorical_encoding(data):
  col = data.name
  if col not in cat_dict:
    cat_dict[col] = {}
  result = []
  for x in data:
    if x not in cat_dict[col] :
      if len(cat_dict[col]) == 0:
        label = 0    
      else:
        label = max(cat_dict[col].values())+1
      cat_dict[col][x] = label
    result.append(cat_dict[col][x])
  return np.array(result)

In [183]:
def preprocessing_data(dataset , onehot=False):
  #name에서 title 추출
  dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.')

  #missing value 처리
  #Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'] => 컬럼은 총 12개
  #print(train.isnull().sum())
  # Age , Cabin, Embarked 3개 컬럼 missing
  # Missing value는 임의의 다른 값으로 fill 처리
  dataset['Age'].fillna(0 , inplace=True) #Age 는 0으로
  dataset['Cabin'].fillna('N00' , inplace=True) #Cabin 은 N00
  dataset['Embarked'].fillna('N' , inplace=True) #Embarked는 N으로

  #categorical data 처리
  # Sex, Ticket, Cabin, Embarked, Title
  cate_col = ['Sex' , 'Ticket' , 'Cabin' , 'Embarked' , 'Title']
  
  for col in cate_col :
    # col_label = train[col].astype('category').cat.codes
    col_label = categorical_encoding(dataset[col])
    if onehot is True:
      num = np.unique(col_label, axis=0).shape[0]
      encoding = np.eye(num)[col_label]
      dataset[col+'_encoding'] = encoding
    else:
      dataset[col+'_encoding'] = col_label

  return dataset


In [184]:
train = preprocessing_data(train)
train.isnull().sum() #missing value 없음
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Sex_encoding,Ticket_encoding,Cabin_encoding,Embarked_encoding,Title_encoding
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,N00,S,Mr,0,0,0,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,1,1,1,1,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,N00,S,Miss,1,2,0,0,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,1,3,2,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,N00,S,Mr,0,4,0,0,0


In [185]:
#실제 학습에 사용할 데이터는 sequential 데이터와 labeling 된 데이터
trainset = train[['PassengerId' , 'Pclass' , 'Age' , 'SibSp' , 'Parch' , 'Fare' , 'Sex_encoding' , 'Ticket_encoding' , 'Cabin_encoding' , 'Embarked_encoding' , 'Title_encoding']]
trainset

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_encoding,Ticket_encoding,Cabin_encoding,Embarked_encoding,Title_encoding
0,1,3,22.0,1,0,7.2500,0,0,0,0,0
1,2,1,38.0,1,0,71.2833,1,1,1,1,1
2,3,3,26.0,0,0,7.9250,1,2,0,0,2
3,4,1,35.0,1,0,53.1000,1,3,2,0,1
4,5,3,35.0,0,0,8.0500,0,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,27.0,0,0,13.0000,0,677,0,0,5
887,888,1,19.0,0,0,30.0000,1,678,146,0,2
888,889,3,0.0,1,2,23.4500,1,614,0,0,2
889,890,1,26.0,0,0,30.0000,0,679,147,1,0


In [186]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from torch.utils.data import Dataset, DataLoader  

In [187]:
device = torch.device('cuda:0')

In [231]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [214]:
trainset = train[['Pclass' , 'Age' , 'SibSp' , 'Parch' , 'Fare' , 'Sex_encoding' , 'Ticket_encoding' , 'Cabin_encoding' , 'Embarked_encoding' , 'Title_encoding']] #to_do : train column add
train_dataloader = DataLoader(dataset=trainset.to_numpy() , batch_size=batch_size, shuffle=False)

In [234]:
#custom dataset
class CustomDataset(Dataset):
  # def __init__(self, data, column):
  def __init__(self, data):
    p_data = preprocessing_data(data)
    self.data = p_data[['Pclass' , 'Age' , 'SibSp' , 'Parch' , 'Fare' , 'Sex_encoding' , 'Ticket_encoding' , 'Cabin_encoding', 'Embarked_encoding' , 'Title_encoding']] #to_do : train column add 
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data.iloc[idx, 0 ] , torch.tensor(self.data.iloc[idx, 1:].to_numpy(),dtype=torch.float, device = device)

In [235]:
test_dataset = CustomDataset(test)
test_dataloader = DataLoader(dataset=test_dataset , batch_size = batch_size , shuffle=False)

In [23]:
def _same_pad(k=1, dil=1):
    # assumes stride length of 1
    # p = math.ceil((l - 1) * s - l + dil*(k - 1) + 1)
    p = math.ceil(dil*(k - 1))
    #print("padding:", p)
    return p

class ResBlock(nn.Module):
    """
        Note To Self:  using padding to "mask" the convolution is equivalent to
        either centering the convolution (no mask) or skewing the convolution to
        the left (mask).  Either way, we should end up with n timesteps.
        Also note that "masked convolution" and "casual convolution" are two
        names for the same thing.
    Args:
        d (int): size of inner track of network.
        r (int): size of dilation
        k (int): size of kernel in dilated convolution (odd numbers only)
        casual (bool): determines how to pad the casual conv layer. See notes.
    """
    def __init__(self, d, r=1, k=3, casual=False, use_bias=False):
        super(ResBlock, self).__init__()
        self.d = d # input features
        self.r = r # dilation size
        self.k = k # "masked kernel size"
        ub = use_bias
        self.layernorm1 = nn.InstanceNorm1d(num_features=2*d, affine=True) # same as LayerNorm
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1x1_1 = nn.Conv1d(2*d, d, kernel_size=1, bias=ub) # output is "d"
        self.layernorm2 = nn.InstanceNorm1d(num_features=d, affine=True)
        self.relu2 = nn.ReLU(inplace=True)
        if casual:
            padding = (_same_pad(k,r), 0)
        else:
            p = _same_pad(k,r)
            if p % 2 == 1:
                padding = [p // 2 + 1, p // 2]
            else:
                padding = (p // 2, p // 2)
        self.pad = nn.ConstantPad1d(padding, 0.)
        #self.pad = nn.ReflectionPad1d(padding) # this might be better for audio
        self.maskedconv1xk = nn.Conv1d(d, d, kernel_size=k, dilation=r, bias=ub)
        self.layernorm3 = nn.InstanceNorm1d(num_features=d, affine=True)
        self.relu3 = nn.ReLU(inplace=True)
        self.conv1x1_2 = nn.Conv1d(d, 2*d, kernel_size=1, bias=ub) # output is "2*d"

    def forward(self, input):
        x = input
        x = self.layernorm1(x)
        x = self.relu1(x)
        x = self.conv1x1_1(x)

        x = self.layernorm2(x)
        x = self.relu2(x)
        x = self.pad(x)
        x = self.maskedconv1xk(x)

        x = self.layernorm3(x)
        x = self.relu3(x)
        x = self.conv1x1_2(x)
        #print("ResBlock:", x.size(), input.size())
        x += input # add back in residual
        return x

class ResBlockSet(nn.Module):
    """
        The Bytenet encoder and decoder are made up of sets of residual blocks
        with dilations of increasing size.  These sets are then stacked upon each
        other to create the full network.
    """
    def __init__(self, d, max_r=16, k=3, casual=False , mt=None):
        super(ResBlockSet, self).__init__()
        self.d = d
        self.max_r = max_r
        self.k = k
        self.mt = mt
        rlist = [1 << x for x in range(15) if (1 << x) <= max_r]
        self.blocks = nn.Sequential(*[ResBlock(d, r, k, casual) for r in rlist])

        # if self.mt =='encoder':
        #   self.avg_pooling = nn.AvgPool1d(3, stride=2)
        # elif self.mt =='decoder':
        #   self.deconv = nn.ConvTranspose1d(d*2 , d*2 , 7, 6)

    def forward(self, input):
        x = input
        x = self.blocks(x)

        
        # if self.mt =='encoder':
        #   x = self.avg_pooling(x)
        
        # elif self.mt =='decoder':
        #   x = self.deconv(x)
     
        return x

class BytenetEncoder(nn.Module):
    """
        d = hidden units
        max_r = maximum dilation rate (paper default: 16)
        k = masked kernel size (paper default: 3)
        num_sets = number of residual sets (paper default: 6. 5x6 = 30 ResBlocks)
        a = relative length of output sequence
        b = output sequence length intercept
    """
    def __init__(self, d=800, max_r=16, k=3, num_sets=6):
        super(BytenetEncoder, self).__init__()
        self.d = d
        self.max_r = max_r
        self.k = k
        self.num_sets = num_sets
        self.pad_in = nn.ConstantPad1d((0, 1), 0.)
        self.conv_in = nn.Conv1d(d, 2*d , 1)
        self.sets = nn.Sequential()
        for i in range(num_sets):
            self.sets.add_module("set_{}".format(i+1), ResBlockSet(d, max_r, k, mt="encoder"))
        # self.conv_out = nn.Conv1d(2*d, 2*d, 1)
        self.conv_out = nn.Conv1d(2*d , 1, 1)

    def forward(self, input):
        x = input
        
        x_len = x.size(-1)
        x = self.conv_in(x)
        x = self.sets(x)
        x = self.conv_out(x)
        x = F.relu(x)
        return x

class SimpleEmbEncoder(nn.Module):
    """
        The decoder only training does not specify how to turn an integer input
        into a 2*d dimensional vector that the decoder network expects, so I
        decided to use a simple embedding network.
    """
    def __init__(self, ne, ed):
        super(SimpleEmbEncoder, self).__init__()
        self.emb = nn.Embedding(num_embeddings=ne, embedding_dim=ed)
        self.ne = ne
    def forward(self, input):
        x = torch.clamp(input, 0, self.ne)
        x = self.emb(x)
        x = x.transpose(1, 2)
        return x

class BytenetDecoder(nn.Module):
    """
        d = hidden units
        max_r = maximum dilation rate (paper default: 16)
        k = masked kernel size (paper default: 3)
        num_sets = number of residual sets (paper default: 6. 5x6 = 30 ResBlocks)
        num_classes = number of output classes (Hunter prize default: 205)
    """
    def __init__(self, d=512, max_r=16, k=3, num_sets=6, num_classes=205,
                 reduce_out=None, use_logsm=True):
        super(BytenetDecoder, self).__init__()
        self.max_r = max_r
        self.k = k
        self.d = d
        self.num_sets = num_sets
        self.use_logsm = use_logsm # this is for NLLLoss
        self.conv_in = nn.Conv1d(1,2*d,1)
        self.sets = nn.Sequential()
        for i in range(num_sets):
            self.sets.add_module("set_{}".format(i+1), ResBlockSet(d, max_r, k, True, mt="decoder"))
            if reduce_out is not None:
                r = reduce_out[i]
                if r != 0:
                    reduce_conv = nn.Conv1d(2*d, 2*d, r, r)
                    reduce_pad = nn.ConstantPad1d((0, r), 0.)
                    self.sets.add_module("reduce_pad_{}".format(i+1), reduce_pad)
                    self.sets.add_module("reduce_{}".format(i+1), reduce_conv)
        self.conv1 = nn.Conv1d(2*d, 2*d, 1)
        self.conv2 = nn.Conv1d(2*d, num_classes, 1)
        #self.logsm = nn.LogSoftmax(dim=1)
        self.fc = nn.Linear(2851, 3401)

    def forward(self, input):
        x = input
        x = self.conv_in(x)
        x = self.sets(x)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        
        # if self.use_logsm:
            # x = self.logsm(x)
        # print(x.shape)
        # x = self.fc(x)
        return x

In [217]:
#@title 기본 제목 텍스트
#hyper para 설정
learning_rate = 0.00001
input_features = trainset.columns.shape[0] -1
max_r = 16
k = 3
num_sets = 6
num_classes = 205
epochs = 1
max_length=3401
feature_num = trainset.columns.shape[0] #9

In [218]:
next(iter(train_dataloader)).shape

torch.Size([32, 10])

In [219]:
encoder = BytenetEncoder(feature_num, max_r, k, num_sets=4).to(device)
decoder = BytenetDecoder(feature_num , max_r, k, num_sets=4, num_classes=feature_num, use_logsm=False).to(device)
params = [{"params": encoder.parameters()}, {"params": decoder.parameters()}]
# criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.Smooth1Loss()
criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(params)


In [220]:
dataset_sizes = len(train)

In [222]:
def train_model(encoder, decoder , criterion, optimizer , scheduler = None , num_epochs=25):
  train_loss_history = []
  for epoch in range(num_epochs):

    encoder.train()
    decoder.train()
    avg_loss = 0.0

    for idx , x in enumerate(train_dataloader):

      encoder.zero_grad()
      decoder.zero_grad()
      optimizer.zero_grad()
      
      x = x.unsqueeze(2).float().to(device)
      output = encoder(x)
      output = decoder(output)

      loss = criterion(output, x)

      loss.backward()
      optimizer.step()

      avg_loss += loss.item()

      print("Epoch {}..... Step: {}/{}........ Average Loss {}".format(epoch,(idx+1)*batch_size , dataset_sizes, avg_loss/(idx+1) ))

      train_loss_history.append(avg_loss/(idx+1)*1000)
  return encoder, decoder, train_loss_history 

In [223]:
_, _ , train_loss_history = train_model(encoder, decoder, criterion, optimizer,None, 5000)

Epoch 0..... Step: 32/891........ Average Loss 7.098460674285889
Epoch 0..... Step: 64/891........ Average Loss 8.263861894607544
Epoch 0..... Step: 96/891........ Average Loss 9.463005542755127
Epoch 0..... Step: 128/891........ Average Loss 10.714524865150452
Epoch 0..... Step: 160/891........ Average Loss 11.91107110977173
Epoch 0..... Step: 192/891........ Average Loss 12.86048165957133
Epoch 0..... Step: 224/891........ Average Loss 14.078989369528633
Epoch 0..... Step: 256/891........ Average Loss 15.210231125354767
Epoch 0..... Step: 288/891........ Average Loss 16.66933955086602
Epoch 0..... Step: 320/891........ Average Loss 18.30506100654602
Epoch 0..... Step: 352/891........ Average Loss 19.429809093475342
Epoch 0..... Step: 384/891........ Average Loss 20.351570804913838
Epoch 0..... Step: 416/891........ Average Loss 21.198547986837532
Epoch 0..... Step: 448/891........ Average Loss 22.240358659199305
Epoch 0..... Step: 480/891........ Average Loss 23.319895458221435
Epoch

KeyboardInterrupt: ignored

In [198]:
encoder(x).squeeze(1).cpu().detach().numpy().shape

(2, 1)

In [226]:
tmp_result = []
passenger_id = []
encoder.eval()
for idx, x in enumerate(train_dataloader):
  # print(x.shape)
  x = x.unsqueeze(2).float().to(device)
  new_x = encoder(x).squeeze(1).cpu().detach().numpy()
  tmp_result.append(new_x)
  # tmp_result.append(encoder(x.unsqueeze(-1).to(device)).squeeze(1).detach().cpu().numpy())
  # passenger_id.append(id)
tmp_result = np.concatenate(tmp_result, axis=0)

In [261]:
#custom dataset
class SurviveDataset(Dataset):
  # def __init__(self, data, column):
  def __init__(self, data):
    p_data = preprocessing_data(data)
    self.data = p_data[['PassengerId' , 'Pclass' , 'Age' , 'SibSp' , 'Parch' , 'Fare' , 'Sex_encoding' , 'Ticket_encoding' , 'Cabin_encoding', 'Embarked_encoding' , 'Title_encoding']] #to_do : train column add 
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data.iloc[idx, 0 ] , torch.tensor(self.data.iloc[idx, 1:].to_numpy(),dtype=torch.float, device = device)

survive_test = SurviveDataset(test)
survive_testsetload = DataLoader(dataset=survive_test , batch_size=batch_size, shuffle=False)

In [262]:
next(iter(survive_testsetload))

[tensor([892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905,
         906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919,
         920, 921, 922, 923]),
 tensor([[  3.0000,  34.5000,   0.0000,   0.0000,   7.8292,   0.0000, 681.0000,
            0.0000,   2.0000,   0.0000],
         [  3.0000,  47.0000,   1.0000,   0.0000,   7.0000,   1.0000, 682.0000,
            0.0000,   0.0000,   1.0000],
         [  2.0000,  62.0000,   0.0000,   0.0000,   9.6875,   0.0000, 683.0000,
            0.0000,   2.0000,   0.0000],
         [  3.0000,  27.0000,   0.0000,   0.0000,   8.6625,   0.0000, 684.0000,
            0.0000,   0.0000,   0.0000],
         [  3.0000,  22.0000,   1.0000,   1.0000,  12.2875,   1.0000, 405.0000,
            0.0000,   0.0000,   1.0000],
         [  3.0000,  14.0000,   0.0000,   0.0000,   9.2250,   0.0000, 685.0000,
            0.0000,   0.0000,   0.0000],
         [  3.0000,  30.0000,   0.0000,   0.0000,   7.6292,   1.0000, 686.0000,
     

In [266]:
test_result = []
pclass_ids = []
for idx, ( p_class , x) in enumerate(survive_testsetload):
  x = x.unsqueeze(2).float().to(device)
  result = encoder(x).squeeze(1).cpu().detach().numpy()
  test_result.append(result)
  pclass_ids.append(p_class)

test_result = np.concatenate(test_result, axis=0)
pclass_ids = np.concatenate(pclass_ids, axis=0)

In [209]:
total = len(test_result)
correct = 0
for inx, x in enumerate(test_result):
  if train.iloc[(np.abs(np.array(tmp_result) - x)).argmin()].Pclass.item() == pclass_result[inx]:
    correct +=1
print(correct/total)

0.69377990430622


In [211]:
np.abs(np.array(tmp_result) - x).argmin()

533

In [212]:
train.iloc[533]

PassengerId                                             534
Survived                                                  1
Pclass                                                    3
Name                 Peter, Mrs. Catherine (Catherine Rizk)
Sex                                                  female
Age                                                       0
SibSp                                                     0
Parch                                                     2
Ticket                                                 2668
Fare                                                22.3583
Cabin                                                   N00
Embarked                                                  C
Title                                                   Mrs
Sex_encoding                                              1
Ticket_encoding                                         119
Cabin_encoding                                            0
Embarked_encoding                       

In [213]:
test.iloc[inx]

PassengerId                              1309
Pclass                                      3
Name                 Peter, Master. Michael J
Sex                                      male
Age                                         0
SibSp                                       1
Parch                                       1
Ticket                                   2668
Fare                                  22.3583
Cabin                                     N00
Embarked                                    C
Title                                  Master
Sex_encoding                                0
Ticket_encoding                           119
Cabin_encoding                              0
Embarked_encoding                           1
Title_encoding                              3
Name: 417, dtype: object

In [272]:
#survive 예측
total = len(test_result)
prediction = []
for inx, x in enumerate(test_result):
  prediction.append(train.iloc[(np.abs(np.array(tmp_result) - x)).argmin()].Survived.item())

In [ ]:
pclass_ids

In [275]:
submission = pd.DataFrame({
        "PassengerId": pclass_ids,
        "Survived": prediction
    })
submission.to_csv('submission.csv', index=False)

In [276]:
!kaggle competitions submit -c titanic -f submission.csv -m "Message"

100% 2.77k/2.77k [00:02<00:00, 1.08kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster